In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE83448"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE83448"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE83448.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE83448.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE83448.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide transcriptional analysis in intestinal biopsies from Crohn's disease (CD) patients."
!Series_summary	"Differential gene expression analysis between CD patients and controls to identify the transcriptional signature that defines the inflamed intestinal mucosa in CD."
!Series_overall_design	"Intestinal biopsy samples were obtained from CD patients and healthy controls. RNA was subsequently extracted from each sample. Gene expression intensities were measured using GE Healthcare/Amersham Biosciences CodeLink Human Whole Genome Bioarray. After performing the gene expression quality control analysis, we characterized the transcriptional profile of the inflamed intestinal mucosa in CD."
Sample Characteristics Dictionary:
{0: ['tissue: intestinal mucosa'], 1: ['inflammation: Control', 'inflammation: Inflamed margin', 'inflammation: Non-inflamed margin']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Analyze gene expression data availability
# From the background info, we can see this is a study with gene expression data from GE Healthcare/Amersham Biosciences CodeLink Human Whole Genome Bioarray
is_gene_available = True

# 2.1 Data Availability
# Looking at the dictionary, we can see that key 1 has inflammation status
# We can use this to infer Crohn's Disease status (inflamed = CD patient, control = healthy control)
trait_row = 1
# Age data is not available in the dictionary
age_row = None
# Gender data is not available in the dictionary
gender_row = None

# 2.2 Data Type Conversion
# Define conversion functions for each variable

def convert_trait(value):
    """
    Convert inflammation status to binary Crohn's Disease indicator.
    0 = No CD (Control), 1 = CD (Inflamed margin or Non-inflamed margin)
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == "Control":
        return 0
    elif value in ["Inflamed margin", "Non-inflamed margin"]:
        return 1
    else:
        return None

# No age data, but define the function as required
def convert_age(value):
    """Convert age to continuous value."""
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# No gender data, but define the function as required
def convert_gender(value):
    """Convert gender to binary value (0=female, 1=male)."""
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save metadata - initial filtering check
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM2203115': [0.0], 'GSM2203116': [0.0], 'GSM2203117': [0.0], 'GSM2203118': [1.0], 'GSM2203119': [1.0], 'GSM2203120': [1.0], 'GSM2203121': [0.0], 'GSM2203122': [1.0], 'GSM2203123': [1.0], 'GSM2203124': [1.0], 'GSM2203125': [0.0], 'GSM2203126': [1.0], 'GSM2203127': [1.0], 'GSM2203128': [1.0], 'GSM2203129': [0.0], 'GSM2203130': [1.0], 'GSM2203131': [1.0], 'GSM2203132': [0.0], 'GSM2203133': [1.0], 'GSM2203134': [1.0], 'GSM2203135': [1.0], 'GSM2203136': [0.0], 'GSM2203137': [1.0], 'GSM2203138': [1.0], 'GSM2203139': [1.0], 'GSM2203140': [0.0], 'GSM2203141': [0.0], 'GSM2203142': [1.0], 'GSM2203143': [1.0], 'GSM2203144': [0.0], 'GSM2203145': [0.0], 'GSM2203146': [0.0], 'GSM2203147': [0.0], 'GSM2203148': [1.0], 'GSM2203149': [1.0], 'GSM2203150': [1.0], 'GSM2203151': [1.0], 'GSM2203152': [1.0], 'GSM2203153': [1.0], 'GSM2203154': [1.0], 'GSM2203155': [1.0], 'GSM2203156': [1.0], 'GSM2203157': [1.0], 'GSM2203158': [1.0], 'GSM2203159': [1.0], 'GSM2203160': [1.0], 'GSM220316

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['GE469557', 'GE469567', 'GE469590', 'GE469632', 'GE469690', 'GE469802',
       'GE469817', 'GE469849', 'GE469866', 'GE469875', 'GE469953', 'GE470103',
       'GE470130', 'GE470157', 'GE470169', 'GE470208', 'GE470218', 'GE470249',
       'GE470296', 'GE470328'],
      dtype='object', name='ID')

Gene data dimensions: 20902 genes × 53 samples


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers
# These identifiers (like GE469557) are not standard human gene symbols
# Standard human gene symbols would be like BRCA1, TP53, etc.
# These look like custom probes/identifiers specific to a microarray platform
# They would need to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE469530', 'GE469548', 'GE469549', 'GE469555', 'GE469557'], 'GB_ACC': ['AI650595.1', 'BU686968.1', 'BU623208.1', 'BE045962.1', 'AY077696.1'], 'Probe_Name': ['GE469530', 'GE469548', 'GE469549', 'GE469555', 'GE469557'], 'Probe_Type': ['DISCOVERY', 'DISCOVERY', 'DISCOVERY', 'DISCOVERY', 'DISCOVERY'], 'DESCRIPTION': ["wa92h11x1 NCI_CGAP_GC6 cDNA clone IMAGE:2303685 3'", "UI-CF-DU1-ado-i-08-0-UIs1 UI-CF-DU1 cDNA clone UI-CF-DU1-ado-i-08-0-UI 3'", "UI-H-FL1-bgd-j-14-0-UI.s1 NCI_CGAP_FL1 cDNA clone UI-H-FL1-bgd-j-14-0-UI 3', mRNA sequence", "hd90g04x4 NCI_CGAP_GC6 cDNA clone IMAGE:2916822 3'", 'clone qd65g07 PRED16 protein (PRED16) mRNA'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Look for alternative gene symbol columns in the annotation data
# First, check if there are any hidden/unprefixed columns that might contain gene symbols
print("Checking for additional columns in annotation data:")
for col in gene_annotation.columns:
    unique_values = gene_annotation[col].dropna().unique()
    if len(unique_values) > 0:
        print(f"Column '{col}' sample: {unique_values[:3]}")

# Since we don't see standard gene symbols, we'll use GB_ACC (GenBank accessions)
# as identifiers for the gene expression data
print("\nUsing GenBank accessions as gene identifiers.")
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GB_ACC')

# Check the mapping dataframe
print(f"\nGene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(preview_df(gene_mapping))

# 3. Convert probe-level measurements to gene expression
# Note: We're working with accession numbers, not gene symbols
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the results
print("\nAfter mapping - Gene data dimensions:", gene_data.shape)
print("\nFirst few gene identifiers after mapping:")
print(gene_data.index[:10])

# Skip normalization since these are not standard gene symbols
# We'll keep the accession numbers as identifiers
print("\nNote: The dataset is using GenBank accessions rather than standard gene symbols.")
print("This may affect downstream analysis that relies on gene symbol annotations.")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Checking for additional columns in annotation data:
Column 'ID' sample: ['GE469530' 'GE469548' 'GE469549']


Column 'GB_ACC' sample: ['AI650595.1' 'BU686968.1' 'BU623208.1']


Column 'Probe_Name' sample: ['GE469530' 'GE469548' 'GE469549']
Column 'Probe_Type' sample: ['DISCOVERY']
Column 'DESCRIPTION' sample: ["wa92h11x1 NCI_CGAP_GC6 cDNA clone IMAGE:2303685 3'"
 "UI-CF-DU1-ado-i-08-0-UIs1 UI-CF-DU1 cDNA clone UI-CF-DU1-ado-i-08-0-UI 3'"
 "UI-H-FL1-bgd-j-14-0-UI.s1 NCI_CGAP_FL1 cDNA clone UI-H-FL1-bgd-j-14-0-UI 3', mRNA sequence"]
Column 'SPOT_ID' sample: ['INCYTE UNIQUE']

Using GenBank accessions as gene identifiers.

Gene mapping dataframe shape: (1156663, 2)
Sample of gene mapping:
{'ID': ['GE469530', 'GE469548', 'GE469549', 'GE469555', 'GE469557'], 'Gene': ['AI650595.1', 'BU686968.1', 'BU623208.1', 'BE045962.1', 'AY077696.1']}



After mapping - Gene data dimensions: (5353, 53)

First few gene identifiers after mapping:
Index(['AA010870', 'AA021186', 'AA029225', 'AA057423', 'AA058586', 'AA127601',
       'AA149620', 'AA150617', 'AA166934', 'AA187037'],
      dtype='object', name='Gene')

Note: The dataset is using GenBank accessions rather than standard gene symbols.
This may affect downstream analysis that relies on gene symbol annotations.

Gene expression data saved to ../../output/preprocess/Crohns_Disease/gene_data/GSE83448.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Skip gene symbol normalization and use the accession numbers directly
print("Processing gene expression data...")
# Don't normalize - these are GenBank accessions, not gene symbols
gene_data_normalized = gene_data  # Use the original gene data with accession numbers

# Save the gene data (without normalization)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")
print(f"Gene data shape: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# 2. Extract clinical features from scratch
print("Extracting clinical features from original clinical data...")
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row,
    convert_trait,
    age_row,
    convert_age,
    gender_row,
    convert_gender
)

# Save the extracted clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

print("Clinical features preview:")
print(preview_df(clinical_features))

# Check if clinical features were successfully extracted
if clinical_features.empty:
    print("Failed to extract clinical features. Dataset cannot be processed further.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Clinical features could not be extracted from the dataset."
    )
    print("Dataset deemed not usable due to lack of clinical features.")
else:
    # 2. Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
    print(f"Linked data shape: {linked_data.shape}")

    # Check if the linked data has gene features
    if linked_data.shape[1] <= 1:
        print("Error: Linked data has no gene features. Dataset cannot be processed further.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=False,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to link gene expression data with clinical features."
        )
    else:
        # 3. Handle missing values systematically
        linked_data = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
        else:
            # 4. Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

            # 5. Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data,
                note="Dataset contains gene expression data for Crohn's Disease patients and healthy controls."
            )

            # 6. Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Processing gene expression data...


Gene expression data saved to ../../output/preprocess/Crohns_Disease/gene_data/GSE83448.csv
Gene data shape: 5353 genes × 53 samples
Extracting clinical features from original clinical data...
Clinical features saved to ../../output/preprocess/Crohns_Disease/clinical_data/GSE83448.csv
Clinical features preview:
{'GSM2203115': [0.0], 'GSM2203116': [0.0], 'GSM2203117': [0.0], 'GSM2203118': [1.0], 'GSM2203119': [1.0], 'GSM2203120': [1.0], 'GSM2203121': [0.0], 'GSM2203122': [1.0], 'GSM2203123': [1.0], 'GSM2203124': [1.0], 'GSM2203125': [0.0], 'GSM2203126': [1.0], 'GSM2203127': [1.0], 'GSM2203128': [1.0], 'GSM2203129': [0.0], 'GSM2203130': [1.0], 'GSM2203131': [1.0], 'GSM2203132': [0.0], 'GSM2203133': [1.0], 'GSM2203134': [1.0], 'GSM2203135': [1.0], 'GSM2203136': [0.0], 'GSM2203137': [1.0], 'GSM2203138': [1.0], 'GSM2203139': [1.0], 'GSM2203140': [0.0], 'GSM2203141': [0.0], 'GSM2203142': [1.0], 'GSM2203143': [1.0], 'GSM2203144': [0.0], 'GSM2203145': [0.0], 'GSM2203146': [0.0], 'GSM2203147': 

Data shape after handling missing values: (53, 5354)

Checking for bias in feature variables:
For the feature 'Crohns_Disease', the least common label is '0.0' with 14 occurrences. This represents 26.42% of the dataset.
The distribution of the feature 'Crohns_Disease' in this dataset is fine.



Linked data saved to ../../output/preprocess/Crohns_Disease/GSE83448.csv
